In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8672.19it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4782.55it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2563092.82it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8703.46it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:03<00:00, 4741.54it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 74731.41it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8688.28it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4788.74it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 56356.03it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 8704.16it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4605.68it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 14629.95it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
hist_bin_size = 30
model = mz.models.DRMM()
model.params['input_shapes'] = [[10,], [10, hist_bin_size,]]
model.params['task'] = ranking_task
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 300
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMM.
Parameter "embedding_trainable" set to True.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [9]:
model.load_embedding_matrix(embedding_matrix)

In [10]:
pred_generator = mz.HistogramDataGenerator(data_pack=predict_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           hist_bin_size=hist_bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, once_every=1, batch_size=len(pred_y))

In [11]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, hist_bin_size, 'LCH',
                                                num_dup=2, num_neg=4, batch_size=20)
len(train_generator)

102

In [12]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 8s 75ms/step - loss: 1.5849
Validation: normalized_discounted_cumulative_gain@3(0): 0.46085027757905045 - normalized_discounted_cumulative_gain@5(0): 0.5360892478728125 - mean_average_precision(0): 0.49095899114018426
Epoch 2/30
102/102 [==============================] - 7s 67ms/step - loss: 1.4984
Validation: normalized_discounted_cumulative_gain@3(0): 0.5740345952020047 - normalized_discounted_cumulative_gain@5(0): 0.6262011011782215 - mean_average_precision(0): 0.5801947873515018
Epoch 3/30
102/102 [==============================] - 11s 106ms/step - loss: 1.3584
Validation: normalized_discounted_cumulative_gain@3(0): 0.5750097489299887 - normalized_discounted_cumulative_gain@5(0): 0.6343800244133054 - mean_average_precision(0): 0.5869700543957561
Epoch 4/30
102/102 [==============================] - 11s 103ms/step - loss: 1.2679
Validation: normalized_discounted_cumulative_gain@3(0): 0.6065906116597288 - normalized_discounted_cum

102/102 [==============================] - 9s 93ms/step - loss: 0.6015
Validation: normalized_discounted_cumulative_gain@3(0): 0.576864812465392 - normalized_discounted_cumulative_gain@5(0): 0.6364934931333227 - mean_average_precision(0): 0.5888049280562754
Epoch 27/30
102/102 [==============================] - 10s 96ms/step - loss: 0.5983
Validation: normalized_discounted_cumulative_gain@3(0): 0.5735865638229847 - normalized_discounted_cumulative_gain@5(0): 0.6349609120733534 - mean_average_precision(0): 0.5889968870090605
Epoch 28/30
101/102 [============================>.] - ETA: 0s - loss: 0.5926Validation: normalized_discounted_cumulative_gain@3(0): 0.5735865638229847 - normalized_discounted_cumulative_gain@5(0): 0.6349609120733534 - mean_average_precision(0): 0.5889968870090605
Epoch 28/30
102/102 [==============================] - 11s 108ms/step - loss: 0.5949
Validation: normalized_discounted_cumulative_gain@3(0): 0.5734217273189437 - normalized_discounted_cumulative_gain@5(0):